In [195]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import cv2
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pdb
import math

def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop

def searchForFocus(filename, substring):
    with open(filename, 'r') as file:
        data = file.read()
        location = data.find(substring)
        croppedStr = data[location+len(substring):]
        # Split at spaces and find first number
        for word in croppedStr.split(): # Split at spaces
            # Delete any commas    
            word = word.replace(',', "")
            try:
                focusPosition = int(word)
                return focusPosition
            except ValueError:
                continue
    file.close()

class Dataset(torch.utils.data.Dataset):
    # ids indicates what subfolders (samples) to access
    def __init__(self, foldername, subfolderPrefix, ids):
        self.foldername = foldername
        self.subfolderPrefix = subfolderPrefix
        self.ids = ids
        
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        sampleFoldername = self.foldername + '/' + self.subfolderPrefix + str(index)
        
        # Load in image
        image = cv2.imread(sampleFoldername + '/before' + str(index) + '.tif', 0)
        
        # Convert to RGB
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        
        # Shift it so each channel is from [0,1]
        image = image * 1 / 255
        
        image = get_random_crop(image, 400, 400)

        # pyTorch expects C X H X W
        image = image.transpose(2, 0, 1)
        
        # Randomly crop to a subset of image (400 X 400) for faster training
        #crop = transforms.RandomCrop(400)
        #image = crop(image)
        
        X = torch.from_numpy(image)
                
        # Normalize according to expect input distribution of resnet
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        
        
        # Randomly flip the image as data augmentation approach, p = 0.5 default
        #hFlip = transforms.RandomHorizontalFlip()
        #vFlip = transforms.RandomVerticalFlip()
        
        #totalTransform = transforms.Compose([normalize, crop])
        
        #X = totalTransform(X)
        
        X = normalize(X)
        
        # Get the label
        beforeFocus = searchForFocus(sampleFoldername + '/focusInfo.txt', 'before focus: ')
        afterFocus = searchForFocus(sampleFoldername + '/focusInfo.txt', 'after focus: ')
        
        y = afterFocus - beforeFocus
        
        
        # Moved to separate createAF file        
#         # Load in image as [0,1] array
#         image = cv2.imread(sampleFoldername + '/before' + str(index) + '.tif', 0) * 1 / 255.0
        
#         # Shift it so is from [-1,1]
#         image *= 2
#         image -= 1
#         X = torch.from_numpy(image)
#         X = X.unsqueeze(0) # Add fake first dimension to specify 1-channel

#         # Get the label
#         beforeFocus = searchForFocus(sampleFoldername + '/focusInfo.txt', 'before focus: ')
#         afterFocus = searchForFocus(sampleFoldername + '/focusInfo.txt', 'after focus: ')
        
#         y = afterFocus - beforeFocus
        
        # X = torch.load(sampleFoldername + '/X')
        # y = torch.load(sampleFoldername + '/y')
        
        return X, y
    

In [196]:
params = {'batch_size': 20,
          'shuffle': True,
          'num_workers': 0}

# Randomly partition the full list into a training set and validation set
numSamples = 150 # total number of samples collected
frac = 1/5 # fraction to be validation
np.random.seed(0)
permutedIds = np.random.permutation(range(numSamples))
splitPoint = int((1-frac) * len(permutedIds))
trainingIds = permutedIds[:splitPoint]
valIds = permutedIds[splitPoint:]

training_set = Dataset('C:/Users/aofeldman/Desktop/AFdataCollection', 'sample', trainingIds)
training_generator = torch.utils.data.DataLoader(training_set, **params)

validation_set = Dataset('C:/Users/aofeldman/Desktop/AFdataCollection', 'sample', valIds)
#validation_generator = torch.utils.data.DataLoader(validation_set, **params)

In [197]:
# X, y = training_set.__getitem__(1)

# def imshow(img,wait):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = np.squeeze(img.numpy())
#     width = int(0.15 * npimg.shape[1])
#     height = int(0.15 * npimg.shape[0])
#     cv2.imshow("Hi",cv2.resize(npimg, (width, height)))
#     cv2.waitKey(wait)
#     cv2.destroyAllWindows()
# imshow(X, 1000)
# print(X.shape)
# print(y)

In [1]:
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/aofeldman/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BasicBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [199]:
# Consider placing dropout layers after conv2d layers (conv2d -> batchnorm2d -> leakyReLU -> dropout(p=0.1))
# And also place after fully connected layers (linear -> leakyReLU -> dropout(p=0.3))
# TODO: Should actually figure out appropriate amount of padding for layers 

fc = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512, 100),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    nn.Linear(100, 10),
    nn.LeakyReLU(negative_slope = 0.1, inplace=True),
    nn.Linear(10, 1)
)

net = resnet18

net.fc = fc

# net = nn.Sequential(
#     # First, let's try to "downsample" by using a large stride with a one-channel convolution
#     nn.Conv2d(1, 1, kernel_size=7, stride=3),
#     nn.Conv2d(1, 4, kernel_size=5, stride=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.MaxPool2d(kernel_size=4, stride=4),
#     nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.MaxPool2d(kernel_size=4, stride=4),
#     nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.MaxPool2d(kernel_size=4, stride=4),
#     nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
#     nn.BatchNorm2d(4),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.MaxPool2d(kernel_size=2, stride=2),

#     nn.Flatten(),
#     nn.Linear(252, 100),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.Linear(100, 10),
#     nn.LeakyReLU(negative_slope = 0.1, inplace=True),
#     nn.Linear(10, 1)
# )

In [200]:
fcLength = len([module for module in fc.modules() if type(module) != nn.Sequential])
netLength = len([module for module in net.modules() if type(module) != nn.Sequential])

layerInd = 0
for p in net.parameters():
    print(p.data.shape)    
    if layerInd < netLength - fcLength:
        p.requires_grad = False
        print('Turned gradient off')
    else:
        p.requires_grad = True
        print('Kept gradient turned on')
    layerInd += 1

torch.Size([64, 3, 7, 7])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64, 64, 3, 3])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64, 64, 3, 3])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64, 64, 3, 3])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64, 64, 3, 3])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([64])
Turned gradient off
torch.Size([128, 64, 3, 3])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128, 128, 3, 3])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128, 64, 1, 1])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128])
Turned gradient off
torch.Size([128, 128, 3

In [201]:
count = 0
for p in net.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
    print(p.data)
print(f'total params: {count}')

tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
            1.7083e-02, -1.2694e-02],
          [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
           -1.2907e-01,  3.7424e-03],
          [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
            2.5632e-01,  6.3573e-02],
          ...,
          [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
           -4.2058e-01, -2.5781e-01],
          [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
            3.9359e-01,  1.6606e-01],
          [-1.3736e-02, -3.6746e-03, -2.4084e-02,  ..., -1.5070e-01,
           -8.2230e-02, -5.7828e-03]],

         [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  ...,  3.2521e-02,
            6.6221e-04, -2.5743e-02],
          [ 4.5687e-02,  3.3603e-02, -1.0453e-01,  ..., -3.1253e-01,
           -1.6051e-01, -1.2826e-03],
          [-8.3730e-04,  9.8420e-02,  4.0210e-01,  ...,  7.0789e-01,
            3.6887e-01,  1.2455e-01],
          ...,
     

tensor([0.1971, 0.1771, 0.1303, 0.1995, 0.1839, 0.0934, 0.2333, 0.2236, 0.1654,
        0.1280, 0.0842, 0.1085, 0.3168, 0.2032, 0.3246, 0.2184, 0.3208, 0.2824,
        0.3408, 0.3339, 0.3307, 0.5571, 0.2821, 0.3081, 0.2114, 0.2971, 0.2361,
        0.5500, 0.1221, 0.3381, 0.1528, 0.1544, 0.1982, 0.0582, 0.1812, 0.2489,
        0.1954, 0.0705, 0.0918, 0.1328, 0.2616, 0.2013, 0.0720, 0.1573, 0.1919,
        0.0813, 0.1170, 0.2504, 0.2863, 0.3032, 0.1476, 0.3696, 0.1870, 0.2097,
        0.1907, 0.2364, 0.1642, 0.1079, 0.2531, 0.1703, 0.1266, 0.0814, 0.2407,
        0.2609, 0.2705, 0.2128, 0.5007, 0.2375, 0.0802, 0.2896, 0.1776, 0.0887,
        0.1094, 0.1834, 0.2812, 0.1971, 0.2021, 0.3443, 0.1411, 0.1362, 0.2676,
        0.1618, 0.2723, 0.2727, 0.2528, 0.0982, 0.4707, 0.2239, 0.3649, 0.1987,
        0.0815, 0.2543, 0.3322, 0.1561, 0.2336, 0.1294, 0.2570, 0.1700, 0.1374,
        0.2215, 0.5015, 0.3132, 0.1487, 0.1174, 0.0916, 0.2130, 0.1393, 0.3057,
        0.5634, 0.1018, 0.0994, 0.0492, 

         [[-0.0080]]]])
tensor([ 0.1694,  0.3368,  0.2993,  0.3745,  0.1513,  0.1781,  0.3167,  0.3947,
         0.1858,  0.2068,  0.1090,  0.2042,  0.2955,  0.0765,  0.2023,  0.2487,
         0.3295,  0.3349,  0.2532,  0.2739,  0.1661,  0.3432,  0.3424,  0.2969,
         0.2226,  0.0993,  0.3328,  0.2349,  0.2894,  0.2296,  0.2719,  0.3945,
         0.1990,  0.2564,  0.2557,  0.3541,  0.1848,  0.2513,  0.3101,  0.2782,
         0.2109,  0.2441,  0.3282,  0.3248,  0.2499,  0.1873,  0.2643,  0.3949,
         0.1962,  0.2587,  0.1708,  0.3381,  0.2238,  0.2498,  0.2787,  0.3783,
         0.3445,  0.2681,  0.2956,  0.1146,  0.2688,  0.3479,  0.1295,  0.2843,
         0.1552,  0.3026,  0.2738,  0.1891,  0.3568,  0.2302,  0.2199,  0.2070,
         0.2119,  0.0971,  0.2482,  0.2264,  0.3555,  0.3113,  0.2386,  0.2654,
         0.2975,  0.2666,  0.2180,  0.1451,  0.2460,  0.1734,  0.2358,  0.2891,
         0.2091,  0.1971,  0.2185,  0.2008,  0.2461,  0.3726,  0.2028,  0.1993,
         0.3652,

In [202]:
import torch.optim as optim

criterion = nn.MSELoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.RMSprop(net.parameters())

In [203]:
# For early stopping, use blocks of epochs (say size 3), and a window (say size 10) of how far to look in the past.
# Then, compare the average loss on the current block to the block in the past. Another idea would be compare the variances.
# If the fraction is not sufficiently small, halt.

max_epochs = 100
# Should be stated as a fraction of the previous error
max_frac = 0.999
window = 10 # How many epochs in the past to compare to
block = 5 # What size block of epochs to use

epoch_training_loss = []
epoch_val_loss = []

for epoch in range(max_epochs):
    print("\nOn epoch: " + str(epoch))
    
    running_loss = 0
    i = 0
    
    net.train()
    for inputs, labels in training_generator:
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(torch.squeeze(outputs), labels.float())
        loss.backward()
        optimizer.step()
        
        print('Outputs', torch.squeeze(outputs))
        print('Batch size: ' + str(len(inputs)))
        running_loss += loss.item() * len(inputs) # Multiply by the batch size
        print('Batch average loss ' + str(loss.item()))
    training_loss = running_loss / training_set.__len__()
    print("Epoch training loss: " + str(training_loss))
    epoch_training_loss.append(training_loss)
    
    with torch.no_grad():
        val_loss = 0
        net.eval()
        for ind in validation_set.ids:
            X, y = validation_set.__getitem__(ind)
            X = X.unsqueeze(0) # Add fake batch dimension
            yHat = net(X.float())
            val_loss += (y - yHat)**2
        val_loss /= validation_set.__len__()
        epoch_val_loss.append(val_loss)
    print("Epoch validation loss: " + str(val_loss))
    
    if len(epoch_val_loss) >= window + block + 1:
        latestBlock = np.mean(epoch_val_loss[-1:-1-block])
        earlierBlock = np.mean(epoch_val_loss[-1-window:-1-window-block])
        
        # latestBlock must be sufficiently smaller than earlierBlock
        if latestBlock / earlierBlock > max_frac:
            print('Converged')
            pdb.set_trace()
            break
            
print('Finished Training')


On epoch: 0
Outputs tensor([-0.3081, -0.3038, -0.2854, -0.2862, -0.2710, -0.2576, -0.2739, -0.2688,
        -0.3055, -0.2413, -0.2815, -0.2743, -0.2689, -0.3155, -0.3266, -0.2784,
        -0.2816, -0.2836, -0.2847, -0.3128], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 77873.671875
Outputs tensor([-114.5681, -107.0456, -107.5486, -104.4761,  -93.4040, -104.5260,
        -106.2856, -104.0927,  -96.6535,  -94.2634, -106.9238, -100.0984,
        -106.1375, -101.9846, -105.1749, -103.9042,  -95.7550, -101.0633,
         -90.0632,  -97.1842], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 73945.796875
Outputs tensor([ -988.4918, -1011.7834,  -961.2576, -1150.4128, -1049.1892, -1068.3984,
         -954.0379, -1171.3805, -1079.7764, -1045.2427, -1123.2295, -1118.2269,
        -1054.9429, -1044.4857,  -937.7114, -1159.4387, -1157.4197, -1023.9839,
        -1044.1715, -1099.8535], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 920752.0
Outputs ten

Epoch validation loss: tensor([[70921.1328]])

On epoch: 4
Outputs tensor([ -63.0962,  -55.5741,  -57.7723,  -57.9302, -192.9997,  -90.4519,
         -74.3669,  -82.3068,  -88.9918,  -87.6199, -208.6293, -185.3494,
         -71.1429, -137.4691,  -84.7143, -182.0281, -154.0600, -102.0044,
        -156.2486, -124.7181], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 58524.71875
Outputs tensor([ -74.8123, -163.7251, -196.1156, -117.4501, -187.4494,  -80.8288,
         -65.5899,  -72.3043,  -98.2622,  -46.8704,  -80.1129, -249.2979,
        -154.5309, -103.5497, -239.2301, -107.1727,  -66.5352, -252.8975,
         -93.9458, -148.5485], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 39648.80859375
Outputs tensor([-104.7965, -189.1286, -190.3096, -227.1131, -119.6425, -218.9950,
         -96.9258,  -80.1604, -137.0454,  -48.8676,  -38.8747,  -65.4113,
        -122.9130,  -68.4709,  -65.0894, -187.3584, -235.9258, -168.1657,
         -56.3085, -135.5975], grad_fn

Epoch validation loss: tensor([[52532.6445]])

On epoch: 8
Outputs tensor([  -1.3896,  -75.6702,  -48.9624, -482.9067,  -17.7339, -187.8246,
        -100.7405, -289.9716,    1.9537,    1.8509,   -2.5025, -408.6460,
          -7.0467, -289.0361,   -2.3640, -121.8157,    3.4328, -274.0158,
        -297.0957,  -67.2784], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 30039.037109375
Outputs tensor([-101.4118, -243.7786, -248.3032, -349.5772, -107.6574, -232.1910,
        -598.8016,  -83.3304, -356.0197, -116.8767, -130.9306,  -15.7841,
         -78.6949,  -30.7212,  -76.0439, -376.4280, -205.3689,  -65.5845,
         -59.8236,  -71.3241], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 74047.3125
Outputs tensor([-213.5966,  -41.1011,   -2.8870,   -3.2117,  -59.4592,    1.6149,
           3.6625,    6.1700,  -65.1441, -325.2875,    9.2202, -267.7357,
           6.0326,    6.7263,    3.9668, -246.4511,    2.6654,    8.6785,
           3.2953,    7.0843], grad_fn

Outputs tensor([   5.7030, -108.9512,  -16.4755,    2.2089,  -85.2866,    3.9034,
        -567.4727, -456.4246, -118.1897, -204.1933,    4.1731, -277.3371,
        -480.7329,    1.4568, -240.4294,   11.0720,    6.6930,  -37.4243,
         -89.9961,  -67.2060], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 47876.3828125
Epoch training loss: 49262.826497395836
Epoch validation loss: tensor([[50714.3047]])

On epoch: 12
Outputs tensor([  -9.0999, -678.0205, -132.0873, -423.6003, -224.2140,  -93.5828,
        -219.0089, -150.3570, -175.0706,  -82.1776, -594.8408,  -12.3742,
         -16.9826,  -64.3456, -216.6940,  -89.9066,  -98.4801, -207.5017,
         -23.6573, -375.7582], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 52780.88671875
Outputs tensor([   7.2625, -160.4923, -165.2971, -159.6828,   -4.2921, -230.1429,
           5.1607,    1.4560, -442.2016,  -60.5089,  -20.9456, -389.9430,
        -361.3826,    9.4086,    2.6087,  -11.6424,   -2.1822,    3.5

Outputs tensor([-273.8638, -266.8656, -511.2007,    6.3639, -532.8373, -250.1325,
          15.2540,   10.2961,    2.8712, -503.8745,    7.0567,   14.9581,
          13.6699,    8.0140,  -34.6981,  -31.6003,   12.5686,    6.8913,
           4.0264,   10.3959], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 25626.322265625
Outputs tensor([  14.9238, -555.4137,   15.2252,    7.9954,  -29.0709,   -1.2775,
           2.0960,    7.7983, -164.2808,    6.3713,   14.9880, -244.8503,
        -487.8528, -129.7473,  -88.7049, -319.3751,    3.9391,    2.7484,
           6.3252,    9.1418], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 32026.90234375
Epoch training loss: 33056.956705729164
Epoch validation loss: tensor([[46460.1914]])

On epoch: 16


...

Outputs tensor([ -11.8509,  -87.6284,  -10.5844, -588.0758,   -8.1475,  -18.9699,
        -612.0756,  -29.3725,   -9.1997,  -11.1891,   -8.9606,  -44.9276,
         -18.4697, -620.0595,  -31.3972,   -4.9462,   -8.5589,  -12.7593,
         -30.0236,   -9.4557], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 47496.30859375
Outputs tensor([ -18.3914,  -13.6826,   -2.7806,  -10.5488, -143.6299, -251.8523,
          -0.5401, -333.0010,  -17.4510,   -4.5308,  -23.4090, -271.6603,
        -123.6572,    3.0597, -428.1378,   -8.7351,  -13.4903,  -12.4941,
          -9.2135,  -14.0287], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 24573.32421875
Outputs tensor([ -13.3288,  -19.4099,  -11.9514,  -10.1598,  -13.1622,  -15.8252,
        -561.1036, -619.0838,   -9.6942, -241.2465, -601.3409, -651.1241,
         -10.4378,  -11.8832,   -9.5605,   -6.9520,  -16.3488,  -59.9689,
         -19.4371,  -10.0810], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 3

Outputs tensor([-457.6409, -299.8934,   -5.9564,    3.6682,   -4.8789,    6.2459,
        -285.6126,   -5.0974,  -10.0404,   -3.8406, -389.7862,    4.0665,
          -1.2229, -371.5650,    1.8375,  -12.5174,    2.4241,    5.2341,
        -402.4168, -138.4644], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 44049.0703125
Outputs tensor([-5.3635e-01, -1.7561e+01, -5.9057e+02, -1.4616e+01, -5.5433e+02,
        -1.5557e+00, -1.4940e+01, -1.2740e+02, -1.1258e+01, -4.3225e+00,
        -3.5204e+00, -2.0697e+00, -6.6786e+02, -2.8854e+02, -2.4394e+01,
        -8.0538e+00, -5.1303e+02, -6.3459e+00, -2.5949e+00, -4.7207e+00],
       grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 32073.693359375
Outputs tensor([-2.9370e+02, -1.2222e+00,  1.4918e+00, -4.2421e+00, -2.3650e+01,
        -5.1139e+02, -4.2881e+02, -1.1361e+01,  1.9798e+00, -1.8672e+02,
        -1.2112e+01, -4.7834e+02,  8.8842e-01, -3.8588e+02, -1.3683e+00,
        -3.2666e-01, -4.9654e+00, -1.6680e+01, -1.

Epoch validation loss: tensor([[38088.5352]])

On epoch: 92
Outputs tensor([-3.5606e+00, -7.2669e+00, -3.6133e+02, -4.9858e+01, -1.1495e+01,
         7.1268e-02, -3.4842e+00, -3.4856e+00, -1.5319e+00, -2.4058e+02,
        -1.1162e+01, -4.4162e+02, -1.1086e+02, -8.2411e+00, -1.2168e+01,
        -2.0129e+02, -9.7604e+00, -1.0819e+00,  3.5580e-02, -9.1076e+01],
       grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 58065.19921875
Outputs tensor([ -14.2181, -582.4659,  -14.2015,  -24.8736,  -40.0302,  -19.3786,
          -7.6444,   -9.2017,  -19.5583, -359.5028,   -9.7844,  -21.6053,
         -72.4018,  -15.4820, -578.8223, -602.9250, -316.8872,   -8.4859,
         -57.0816,  -11.0148], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 24574.11328125
Outputs tensor([ -23.3782, -180.0767,  -12.9581, -542.4366,  -45.9004, -288.6250,
          -8.8164, -405.0290,  -16.2969, -564.0381, -325.5338,  -57.2025,
         -12.1749,   -9.1964,  -12.2407, -481.7052,   -7.7681

Outputs tensor([ -14.4181,  -20.2429,  -98.2420, -381.2557,  -10.1395,  -10.4861,
        -546.9937,  -20.3020, -734.3074, -555.3887,  -15.7143,  -17.4608,
         -12.3170,  -52.7166,  -40.2205,  -12.5629,  -12.0424,  -13.7131,
        -198.0040,  -10.8737], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 36324.1328125
Epoch training loss: 31968.575520833332
Epoch validation loss: tensor([[37174.0273]])

On epoch: 96
Outputs tensor([-310.4468,  -19.3008,   -7.6207,   -8.0755, -403.7179, -391.2567,
          -6.6707,  -11.0460,  -17.7113, -391.0448,  -12.4871,  -13.9782,
         -10.2741,   -6.7063,   -5.9610,   -3.8281,   -6.7522,   -4.7431,
        -364.6257, -377.8505], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 37847.8515625
Outputs tensor([-404.1747,  -15.1962, -434.3787,  -10.1639,   -9.4416,  -23.3110,
          -9.9176,  -29.7034,  -36.0713,  -21.9454, -486.1861, -593.5092,
          -9.6355,  -10.1306, -425.1119,  -14.1400, -571.3459,   -8.89

Outputs tensor([ -12.1465, -568.1345,  -17.6258,  -11.6241,  -14.1585,  -12.6436,
         -36.0855,  -67.7314,  -15.5839, -127.8419,  -15.2917,  -72.8548,
          -7.3263, -342.1699,  -20.3834,  -30.1604,  -59.5917,  -11.2740,
        -286.8084, -578.5343], grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 39872.0625
Outputs tensor([-1.7319e+00, -9.3398e+00, -1.0478e+01, -5.4206e-01, -9.8374e+00,
        -4.9045e+02, -2.8164e+01, -2.5409e+02, -1.0812e+01, -7.9066e+00,
        -1.9128e+01, -2.0946e+01,  3.3273e-01, -4.3500e+00, -1.1233e-01,
        -5.5376e+02, -1.3493e+01, -1.1510e+01, -1.8014e+00, -9.2833e+00],
       grad_fn=<SqueezeBackward0>)
Batch size: 20
Batch average loss 31269.375
Epoch training loss: 38904.185872395836
Epoch validation loss: tensor([[32595.1152]])
Finished Training


In [207]:
with torch.no_grad():
    for i, dataset in enumerate([training_set, validation_set]):
        if i == 0:
            print('\nTraining Set Results')
        else:
            print('\nValidation Set Results')
        MSE = 0
        avgAbsDev = 0
        for sample in dataset.ids:
            X, y = dataset.__getitem__(sample)
            X = X.unsqueeze(0) # Add fake batch dimension
            net.eval()
            yHat = net(X.float()).numpy()
            MSE += (yHat - y)**2
            avgAbsDev += np.abs(yHat - y)

            print('Sample: ' + str(sample))
            print('y: ' + str(y))
            print('yHat: ' + str(yHat))
        MSE /= len(dataset.ids)
        avgAbsDev /= len(dataset.ids)
        print('RMSE on dataset: ' + str(np.sqrt(MSE)))
        print('Avg Abs Dev on dataset: ' + str(avgAbsDev))


Training Set Results
Sample: 114
y: -457
yHat: [[-553.08344]]
Sample: 62
y: 259
yHat: [[-81.724304]]
Sample: 33
y: -159
yHat: [[-2.3037307]]
Sample: 107
y: -278
yHat: [[-6.502552]]
Sample: 7
y: 45
yHat: [[-8.704995]]
Sample: 100
y: 290
yHat: [[-18.260115]]
Sample: 40
y: 17
yHat: [[-12.659433]]
Sample: 86
y: 288
yHat: [[-113.88098]]
Sample: 76
y: -132
yHat: [[-3.7797906]]
Sample: 71
y: -611
yHat: [[-511.12024]]
Sample: 134
y: 84
yHat: [[-11.00695]]
Sample: 51
y: -465
yHat: [[-373.11786]]
Sample: 73
y: -361
yHat: [[-12.095773]]
Sample: 54
y: 70
yHat: [[-5.6661673]]
Sample: 63
y: -61
yHat: [[-13.47722]]
Sample: 37
y: -538
yHat: [[-508.8604]]
Sample: 78
y: -361
yHat: [[-277.37064]]
Sample: 90
y: -343
yHat: [[-378.21066]]
Sample: 45
y: 300
yHat: [[-38.641758]]
Sample: 16
y: -525
yHat: [[-437.4524]]
Sample: 121
y: -622
yHat: [[-586.09863]]
Sample: 66
y: -259
yHat: [[-0.9929946]]
Sample: 24
y: -604
yHat: [[-537.6856]]
Sample: 8
y: -362
yHat: [[-159.48254]]
Sample: 126
y: 14
yHat: [[-15.99031

In [ ]:
%matplotlib qt5
plt.figure()
plt.plot(range(max_epochs), epoch_training_loss, range(max_epochs), epoch_val_loss)
plt.legend(['Training', 'Validation'])

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in resnet18.state_dict():
    print(param_tensor, "\t", resnet18.state_dict()[param_tensor].size())

In [ ]:
resnet18.fc